In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf


In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()


x_train = x_train / 255
x_test = x_test / 255



x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [ ]:
#!pip uninstall keras-tuner -y
!pip install keras-tuner -q


In [ ]:
import keras_tuner
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout

hp = keras_tuner.HyperParameters()

def build_model(hp):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
  #number of layers
  for i in range(hp.Int("num_layers", 2,8)):
    model.add(tf.keras.layers.Dense(
        units=hp.Int(f"units_{i}", min_value=16, max_value=512, step=16),
        activation=hp.Choice("activation", ["softplus", "relu", "tanh", "sigmoid", "elu"])
    ))

    if hp.Choice("use_dropout", [True, False]):
      model.add(tf.keras.layers.Dropout(
          hp.Float(f"dropout_rate_{i}", min_value=0.0, max_value=0.3, step=0.1)
      ))
  model.add(tf.keras.layers.Dense(10, activation = "softmax"))

  optimizer = hp.Choice("optimizer",["adam","nadam","rmsprop","sgd", "adamax", "adagrad"])
  learning_rate = hp.Float("learning_rate", min_value= 1e-5, max_value=1e-2, sampling="log")

  if optimizer == "adam":
    optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
  if optimizer == "nadam":
    optimizer = tf.keras.optimizers.Nadam(learning_rate = learning_rate)
  if optimizer == "rmsprop":
    optimizer = tf.keras.optimizers.RMSprop(learning_rate = learning_rate)
  if optimizer == "sgd":
    optimizer = tf.keras.optimizers.SGD(learning_rate = learning_rate)
  if optimizer == "adamax":
    optimizer = tf.keras.optimizers.Adamax(learning_rate = learning_rate)
  if optimizer == "adagrad":
    optimizer = tf.keras.optimizers.Adagrad(learning_rate = learning_rate)

  # Choosing optimizer and learning rate
  #optimizer_name = hp.Choice("optimizer", ["adam", "nadam", "rmsprop", "sgd", "adamax", "adagrad"])
  #learning_rate = hp.Float("learning_rate", min_value=1e-5, max_value=1e-2, sampling="log")

  # Mapping optimizer names to their respective classes
  #optimizer_classes = {
  #   "adam": tf.keras.optimizers.Adam,
  #  "nadam": tf.keras.optimizers.Nadam,
  # "rmsprop": tf.keras.optimizers.RMSprop,
  # "sgd": tf.keras.optimizers.SGD,
  # "adamax": tf.keras.optimizers.Adamax,
  # "adagrad": tf.keras.optimizers.Adagrad
  #}

  # Creating the optimizer instance
  #optimizer = optimizer_classes[optimizer_name](learning_rate=learning_rate)
  model.compile(
      loss = tf.keras.losses.SparseCategoricalCrossentropy(),
      optimizer = optimizer,
      metrics = [tf.keras.metrics.SparseCategoricalAccuracy(), "accuracy"]
  )

  hp.Choice("batch_size", [32, 64,128, 256, 512])



  return model


early_stopping = EarlyStopping(
    monitor = "val_accuracy",
    patience = 3,
    restore_best_weights = True

)

In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=25,
    executions_per_trial=2,
    overwrite=True,
    directory="/Users/alexandr/Desktop/HomeTask_12",
    project_name="ther_bast_parametrs",
)

tuner.search_space_summary()

Search space summary
Default search space size: 10
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 8, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 512, 'step': 16, 'sampling': 'linear'}
activation (Choice)
{'default': 'softplus', 'conditions': [], 'values': ['softplus', 'relu', 'tanh', 'sigmoid', 'elu'], 'ordered': False}
use_dropout (Choice)
{'default': 1, 'conditions': [], 'values': [1, 0], 'ordered': True}
dropout_rate_0 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.3, 'step': 0.1, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 512, 'step': 16, 'sampling': 'linear'}
dropout_rate_1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.3, 'step': 0.1, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'nadam', 'rmsprop', 'sgd', 'adam

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

#tuner.search(x_train, y_train, epochs=50, validation_data=(x_test, y_test), callbacks=[early_stopping, reduce_lr])


tuner.search(
    x=x_train,
    y=y_train,
    epochs=35,
    validation_data=(x_test, y_test),
    callbacks = [early_stopping]
)

Trial 25 Complete [00h 09m 00s]
val_accuracy: 0.8789499998092651

Best val_accuracy So Far: 0.8942999839782715
Total elapsed time: 07h 02m 31s


In [ ]:
# Отримання найкращої моделі
#best_model = tuner.get_best_models(num_models=1)[0]
#best_model.summary()

# Тренування та оцінка моделі
#best_model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), callbacks=[early_stopping, reduce_lr])

#test_loss, test_acc = best_model.evaluate(x_test, y_test)
#print('Test accuracy:', test_acc)
#print('Test loss:', test_loss)


best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best parameters:")
print(f"Number of layers: {best_hps.get('num_layers')}")
for i in range(best_hps.get('num_layers')):
  print(f"Number of neurons in the layer {i+1}: {best_hps.get(f'units_{i}')}")
print(f"Activation function:{best_hps.get('activation')}")
print(f"Optimizer: {best_hps.get('optimizer')}")
if best_hps.get("use_dropout"):
    # Проверяем, существует ли 'dropout_rate'
    if 'dropout_rate' in best_hps.values:
        dropout_rate = best_hps.get('dropout_rate')
        print(f"Dropout rate: {dropout_rate}")
    else:
        print("Dropout rate was not selected.")
else:
    print("Dropout is not used.")


Best parameters:
Number of layers: 5
Number of neurons in the layer 1: 512
Number of neurons in the layer 2: 480
Number of neurons in the layer 3: 48
Number of neurons in the layer 4: 496
Number of neurons in the layer 5: 96
Activation function:softplus
Optimizer: adam
Dropout rate was not selected.


In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
for i in range(best_hps.get('num_layers')):
  model.add(tf.keras.layers.Dense(
      units = best_hps.get(f'units_{i}'),
      activation = best_hps.get('activation')
      ))
model.add(tf.keras.layers.Dense(19, activation = "softmax"))

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = optimizer,
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy(), "accuracy"]

)

batch_size = best_hps.get('batch_size')

history = model.fit(
    x_train,
    y_train,
    epochs =50,
    batch_size = batch_size,
    validation_data = (x_test, y_test),
    #callbacks = [early_stopping]

)

print(f"Final training accuracy: {history.history['accuracy'][-1]}")
print(f"Final validation accuracy: {history.history['val_accuracy'][-1]}")

Epoch 1/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.5891 - loss: 1.1655 - sparse_categorical_accuracy: 0.5891 - val_accuracy: 0.8159 - val_loss: 0.4964 - val_sparse_categorical_accuracy: 0.8159
Epoch 2/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/step - accuracy: 0.8342 - loss: 0.4485 - sparse_categorical_accuracy: 0.8342 - val_accuracy: 0.8359 - val_loss: 0.4537 - val_sparse_categorical_accuracy: 0.8359
Epoch 3/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 11s 43ms/step - accuracy: 0.8549 - loss: 0.3966 - sparse_categorical_accuracy: 0.8549 - val_accuracy: 0.8459 - val_loss: 0.4282 - val_sparse_categorical_accuracy: 0.8459
Epoch 4/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 10s 44ms/step - accuracy: 0.8660 - loss: 0.3625 - sparse_categorical_accuracy: 0.8660 - val_accuracy: 0.8537 - val_loss: 0.4054 - val_sparse_categorical_accuracy: 0.8537
Epoch 5/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 20s 41ms/step - accuracy: 0.8694 - loss: 0.3526 - sparse_categorical_accuracy: 0.8694 - val_accuracy: 0.8596 - val_loss: